# Hair in the wind

In [1]:
from IPython.core.display import HTML
css_file = 'https://raw.githubusercontent.com/ngcm/training-public/master/ipython_notebook_styles/ngcmstyle.css'
HTML(url=css_file)

In this coursework the Boundary Value Problem of multiple hairs on a sphere (head) is solved.
A major use of numerical methods is in the creation of computer graphics in films, games, and other media. The generation of “realistic” animations of, for example, smoke, or water, or explosions, are typical problems. One case that remains difficult is the simulation of hair, particularly under external forces such as wind and rain.

The locations of the hairs are given in $(x, y, z)$ by:

\begin{align}
    x(s) =& R\cos(\theta(s))\cos(\phi)\\
    y(s) =& -R\cos(\theta(s))\sin(\phi)\\
    x(s) =& R\sin(\theta(s))
\end{align}

In [59]:
from __future__ import division
import numpy as np
from numpy import cos, sin, exp, log, pi
from scipy.integrate import odeint, quad
from scipy.optimize import brentq
from matplotlib import pyplot as plt
from numba import jit

from IPython.display import display, Math, Latex
from matplotlib import animation
from JSAnimation import IPython_display

from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16
rcParams['figure.figsize'] = (12,6)
%matplotlib inline

In [29]:
def d8ds(theta, s, phi, fx, fg=0.2):
    assert type(theta) == np.ndarray,\
    "Theta should be a numpy array"
    assert theta.ndim == 1,\
    "Theta should be a vector"
    assert type(s) and type(phi) and type(fx) == float,\
    "s, phi, fg and fx should be floats"
    
    d8_ds = np.zeros_like(theta)
    d8_ds[0] = theta[1]
    d8_ds[1] = s*fg*cos(theta[0]) + s*fx*cos(phi)*sin(theta[0])
    return d8_ds

In [39]:
def shoot(z0, s, theta, phi, fx, fg=0.2):
    #assert type(d8ds) == function,\
    #"d8ds (the derivative of theta with respect to s) must be a function"
    T8 = np.array([theta, z0])
    minz = odeint(d8ds, T8, s, args=(phi, fx, fg))
    #print z
    return minz[-1, 1]

In [53]:
z0

6.1873720776544e-10

In [74]:
def hair_pos(R, L, fx, thL, phiL=[0.0]):
    thL = np.array(thL)
    if len(phiL) > 1:
        phiL = np.array(phiL)
        assert len(thL) == len(phiL),\
        "Phi and Theta must be of equal size"
        assert phiL.ndim == 1,\
        "Phi must be a vector"
    else:
        phiL = np.ones_like(thL) * phiL
    assert thL.ndim == 1,\
    "Theta must be a vector"
    N = 5
    s = np.linspace(0,L,N)[::-1]
    x = np.zeros((len(thL),N))
    y = np.zeros((len(thL),N))
    z = np.zeros((len(thL),N))
    for i in np.arange(len(thL)):
        msolve = brentq(shoot, L, -L, args=(s, thL[i], phiL[i], fx, fg))
        thetas = odeint(d8ds, np.array([thL[i], msolve]), s, args=(phiL[i], fx, fg))
        print thetas[:, 0]
        xfun = lambda s: cos(thetas[:, 0]) * cos(phi) + fx * sin(phi)
        print xfun(s)
        yfun = lambda s: -cos(thetas[:, 0]) * sin(phi) + fx * cos(phi)
        zfun = lambda s: sin(thetas[:, 0])
        x[i, :] = quad(xfun, 0, L)

In [75]:
R = 10.0
L = 4.0
fx = 0.0
phi = 0.0
hairs = 10
fg = 0.2
thL = np.linspace(-pi/2.0, pi/2.0, hairs)
s = np.linspace(0, L, 100)[::-1]
x = hair_pos(R, L, fx, thL)

[-1.57079633 -1.57079632 -1.57079632 -1.57079632 -1.57079632]
[  6.12323400e-17   1.98676726e-09   3.97608153e-09   5.96539602e-09
   7.95471029e-09]


error: Supplied function does not return a valid float.

In [11]:
type(d8ds)

function

In [58]:
len([0.0])

1